# This file reads silver layer from adls gen2 and proccess it then save it as gold layer in aws db
It should run once every x moment
It is schudeled using databricks cron jobs jo run every hour

In [0]:
from pyspark.sql.window import Window

# ======= ADLS Gen2 Config =======
storage_account = "xxx"
container = "xxx" 
client_id = "xxx"
tenant_id = "xxx"
client_secret = "xxx"

# ======= Spark Configuration for OAuth =======
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(
    f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider"
)
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(
    f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
    f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
)

adls_base_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/"
target_path = f"{adls_base_path}gold/customers/"
checkpoint_path = f"{adls_base_path}checkpoints/customers_delta/"


# ======= AWS Config =======

jdbc_url = "jdbc:postgresql://mlprojectdb.cx4k04y48cjc.eu-north-1.rds.amazonaws.com:5432/telco_churn"

db_properties = {
    "user": "postgres",
    "password": "mlpipeline",
    "driver": "org.postgresql.Driver"
}

silver_path = f"{adls_base_path}/silver/customers"

In [0]:
from pyspark.sql.functions import col, row_number
df = spark.read.format("delta").load(silver_path)


window = Window.partitionBy("customer_id").orderBy(col("updated_at").desc())

gold_df = (
    df
    .withColumn("rn", row_number().over(window))
    .filter(col("rn") == 1)
    .drop("rn", "year", "month", "day", "hour")
)

staging_table = "customers_staging"

gold_df.write \
    .jdbc(
        url=jdbc_url,
        table=staging_table,
        mode="overwrite",
        properties=db_properties
    )


In [0]:
cols_to_drop = ["event_time", "year", "month", "day", "hour", "minute"]

existing_ids_df = spark.read.jdbc(jdbc_url, "customers", properties=db_properties).select("customer_id")
new_customers_only = gold_df.join(existing_ids_df, on="customer_id", how="left_anti")
new_customers_only = new_customers_only.drop(*cols_to_drop)

new_customers_only.write.mode("append").jdbc(jdbc_url, "customers", properties=db_properties)